In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.exceptions import RequestException
import random

In [8]:
keyword = "Canon_RF_100_500"

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}
data = []

def fetch_page(url, headers, retries=3, backoff_factor=0.3):


    """
    Fetches the content of a webpage with retry and backoff logic.

    This function sends a GET request to the specified URL and retries the request if the
    request fails or encounters a 503 Service Unavailable error. The retry behavior includes
    exponential backoff with jitter to prevent overwhelming the server.

    Args:
        :param url (str): The URL of the webpage to fetch.
        :param headers (dict): A dictionary of HTTP headers to send with the request.
        :param retries (int, optional): The number of retry attempts for failures (default is 3).
        :param backoff_factor (float, optional): The base delay in seconds for exponential backoff (default is 0.3).

    :return
        requests.Response or None: The HTTP response object if the request is successful and
        returns status code 200, otherwise `None` if all retries fail.

    Notes:
        - This function requires the `requests` and `random` libraries.
        - If retries are exhausted, or a non-retriable error occurs, the function returns `None`.
        - Status code 503 triggers a retry with exponential backoff.

    """
    
    session = requests.Session()
    for retry in range(retries):
        try:
            result = session.get(url, headers=headers)
            if result.status_code == 200:
                return result
            elif result.status_code == 503:
                print(f"503 Service Unavailable. Retrying in {backoff_factor} seconds...")
                time.sleep(backoff_factor)
                backoff_factor *= 2*(1+ random.random())  # Exponential backoff
            else:
                print(f"Failed to retrieve page: Status code {result.status_code}")
                break
        except RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(backoff_factor)
            backoff_factor *= 2*(1+ random.random())  # Exponential backoff
    return None
# 
# for page_number in range(1, 6):
#     url = f"https://www.ebay.com/sch/i.html?_nkw={keyword}&_pgn={page_number}&_ipg=240"
#     result = fetch_page(url, headers)
# 
#     if result:
#         soup = BeautifulSoup(result.content, 'html.parser')
#         listings = soup.find_all('li', attrs={'class': 's-item'})
# 
#         for listing in listings:
#             listing_dict = {}
# 
#             title = listing.find('div', attrs={'class': 's-item__title'})
#             if title:
#                 listing_dict['Title'] = title.get_text()
#             else:
#                 listing_dict['Title'] = None
# 
#             price = listing.find('span', attrs={'class': 's-item__price'})
#             if price:
#                 listing_dict['Price'] = price.get_text()
#             else:
#                 listing_dict['Price'] = None
# 
#             url = listing.find('a', attrs={'class': 's-item__link'})
#             if url:
#                 listing_dict['URL'] = url['href']
#             else:
#                 listing_dict['URL'] = None
# 
#             shipping = listing.find('span', attrs={'class': "s-item__shipping s-item__logisticsCost"})
#             if shipping:
#                 listing_dict['Shipping'] = shipping.get_text()
#             else:
#                 listing_dict['Shipping'] = None
# 
#             location = listing.find('span', attrs={'class': 's-item__location s-item__itemLocation'})
#             if location:
#                 listing_dict['Location'] = location.get_text()
#             else:
#                 listing_dict['Location'] = None
# 
#             condition = listing.find('span', attrs={'class': 'SECONDARY_INFO'})
#             if condition:
#                 listing_dict['Condition'] = condition.get_text()
#             else:
#                 listing_dict['Condition'] = None
# 
#             listing_type = listing.find('span', attrs={'class':'s-item__purchase-options s-item__purchaseOptions'})
#             if listing_type:
#                 listing_dict['Listing Type'] = listing_type.get_text()
#             else:
#                 listing_dict['Listing Type'] = None
# 
#             data.append(listing_dict)
#     else:
#         print(f"Failed to retrieve page {page_number} after multiple attempts.")

# # Create DataFrame
# df = pd.DataFrame(data)
# 
# # Check the DataFrame to ensure data has been scraped
# print(df.head())
# 
# # Save DataFrame to CSV
# df.to_csv(f"{keyword}_ebay_listings.csv", index=False)

In [4]:
print(help(fetch_page))

Help on function fetch_page in module __main__:

fetch_page(url, headers, retries=3, backoff_factor=0.3)
    Fetches the content of a webpage with retry and backoff logic.
    
    This function sends a GET request to the specified URL and retries the request if the
    request fails or encounters a 503 Service Unavailable error. The retry behavior includes
    exponential backoff with jitter to prevent overwhelming the server.
    
    Args:
        :param url (str): The URL of the webpage to fetch.
        :param headers (dict): A dictionary of HTTP headers to send with the request.
        :param retries (int, optional): The number of retry attempts for failures (default is 3).
        :param backoff_factor (float, optional): The base delay in seconds for exponential backoff (default is 0.3).
    
    :return
        requests.Response or None: The HTTP response object if the request is successful and
        returns status code 200, otherwise `None` if all retries fail.
    
    Not

In [9]:
df = pd.read_csv("Canon_RF_100_500_ebay_listings.csv")

In [10]:

# Define the keyword to filter by
keywords = keyword.split('_')
# Filter the DataFrame to include only rows where the keyword appears in relevant columns
filtered_df_all_keywords = df
for kw in keywords:
    filtered_df_all_keywords = filtered_df_all_keywords[filtered_df_all_keywords['Title'].str.contains(kw, case=False, na=False)]

# Function to clean and convert price to a float
def clean_price(price):
    if 'to' in price:
        price = price.split('to')[0].strip()
    return float(price.replace('$', '').replace(',', '').strip())

# Apply the clean_price function to the 'Price' column
filtered_df_all_keywords['Price'] = filtered_df_all_keywords['Price'].apply(clean_price)

# Filter the DataFrame to include only rows where the price is greater than or equal to 1500
filtered_df_all_keywords = filtered_df_all_keywords[filtered_df_all_keywords['Price'] >= 1000]

In [11]:
filtered_df_all_keywords.to_csv(f"{keyword}_filtered_ebay_listings.csv", index=False)